In [2]:
!pip install tinydb

import paho.mqtt.client as mqtt
import json
from tinydb import TinyDB, Query
from datetime import datetime, UTC
import pandas as pd
import matplotlib.pyplot as plt

# MQTT settings
broker = "broker.hivemq.com"
port = 1883
topic = "sit225/gyro-data"

# TinyDB setup
db = TinyDB('gyro_data_tinydb.json')  # Creates a JSON file for storage
collection = db.table('readings')

def on_connect(client, userdata, flags, rc, properties=None):
    print("Connected with result code", rc)
    client.subscribe(topic)

def on_message(client, userdata, msg):
    try:
        data = json.loads(msg.payload.decode())
        data['timestamp'] = datetime.now(UTC).isoformat()  # Store as ISO format string
        collection.insert(data)
        print("Data inserted to TinyDB:", data)
    except Exception as e:
        print("Error:", e)

# Create MQTT client and connect
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
client.on_connect = on_connect
client.on_message = on_message

print("Starting MQTT client for TinyDB. Run for 30 minutes to collect data...")
client.connect(broker, port, 60)
client.loop_start()

# Let it run for 30 minutes (1800 seconds) to collect data
import time
time.sleep(1800)

# Stop the MQTT client after data collection
client.loop_stop()
client.disconnect()
print("Data collection completed. Processing data...")

# Data processing and visualization for TinyDB
def process_tinydb_data():
    # Retrieve all documents from TinyDB
    all_data = collection.all()
    
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    
    # Convert timestamp string back to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    print(f"Exported {len(df)} rows from TinyDB")
    
    # Clean the data
    for col in ["x", "y", "z"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
    before = len(df)
    df = df.dropna(subset=["x", "y", "z"]).reset_index(drop=True)
    
    print(f"Removed {before - len(df)} bad rows → {len(df)} cleaned rows remain")
    
    # Save cleaned CSV
    df.to_csv("clean_tinydb.csv", index=False)
    print("Saved clean_tinydb.csv")
    
    # Generate visualizations
    # Plot x, y, z separately
    for axis in ["x", "y", "z"]:
        plt.figure(figsize=(10, 4))
        plt.plot(df["timestamp"], df[axis])
        plt.title(f"Gyroscope {axis} over time (TinyDB)")
        plt.xlabel("Time")
        plt.ylabel("dps")
        
        # Format time axis
        plt.gcf().autofmt_xdate()   # auto rotate + space labels
        plt.tight_layout()
        
        plt.savefig(f"{axis}_tinydb.png")
        plt.close()
    
    # Combined plot
    plt.figure(figsize=(10, 4))
    plt.plot(df["timestamp"], df["x"], label="x")
    plt.plot(df["timestamp"], df["y"], label="y")
    plt.plot(df["timestamp"], df["z"], label="z")
    plt.title("Gyroscope x/y/z over time (TinyDB)")
    plt.xlabel("Time")
    plt.ylabel("dps")
    plt.legend()
    
    plt.gcf().autofmt_xdate()
    plt.tight_layout()
    
    plt.savefig("xyz_combined_tinydb.png")
    plt.close()
    
    print("Visualizations created for TinyDB data")
    return df

# Process the data
df_tinydb = process_tinydb_data()

# Show basic statistics
print("\nTinyDB Data Statistics:")
print("=======================")
print(f"Total records: {len(df_tinydb)}")
for axis in ["x", "y", "z"]:
    print(f"{axis}-axis - Min: {df_tinydb[axis].min():.2f}, Max: {df_tinydb[axis].max():.2f}, Mean: {df_tinydb[axis].mean():.2f}")

# Close the database connection
db.close()

Starting MQTT client for TinyDB. Run for 30 minutes to collect data...
Connected with result code Success
Data inserted to TinyDB: {'x': 14.04, 'y': -9.4, 'z': -4.7, 'timestamp': '2025-09-02T19:08:39.888832+00:00'}
Data inserted to TinyDB: {'x': 2.2, 'y': -2.14, 'z': -2.32, 'timestamp': '2025-09-02T19:08:40.917881+00:00'}
Data inserted to TinyDB: {'x': -2.44, 'y': -0.98, 'z': 0.0, 'timestamp': '2025-09-02T19:08:41.937938+00:00'}
Data inserted to TinyDB: {'x': 1.1, 'y': -3.42, 'z': -1.28, 'timestamp': '2025-09-02T19:08:42.958641+00:00'}
Data inserted to TinyDB: {'x': 4.09, 'y': -0.73, 'z': 2.75, 'timestamp': '2025-09-02T19:08:43.988562+00:00'}
Data inserted to TinyDB: {'x': 43.03, 'y': 19.04, 'z': 19.71, 'timestamp': '2025-09-02T19:08:45.008398+00:00'}
Data inserted to TinyDB: {'x': -18.01, 'y': 7.51, 'z': 17.27, 'timestamp': '2025-09-02T19:08:46.038105+00:00'}
Data inserted to TinyDB: {'x': -0.55, 'y': 0.18, 'z': 0.06, 'timestamp': '2025-09-02T19:08:47.058050+00:00'}
Data inserted to T